In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from scipy.stats.mstats import winsorize
from sklearn.preprocessing import StandardScaler  # 표준화 패키지 라이브러리 
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

## XGBoost classifier
사이킷런 래퍼 XGBoost 모듈 파라미터 기준

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_set, y_set, test_size=0.2, random_state=156)

In [ ]:
# 파이썬 래퍼와 동일한 하이퍼 파라미터 수치
# 트리 100개만 생성
xgb_wrapper = XGBClassifier(n_estimators= 100, learning_rate = 0.1, max_depth = 3)
#xgb_wrapper = XGBClassifier(n_estimators= 100, learning_rate = 0.1, max_depth = 3,objective= "multi:softmax")
xgb_wrapper.fit(X_train, y_train, early_stopping_rounds= 50, eval_metric='logloss')

xgb_preds = xgb_wrapper.predict(X_test)
xgb_pred_probs = xgb_wrapper.predict_proba(X_test)[:,1]


from sklearn.metrics import roc_auc_score \
from sklearn.metrics import f1_score \
from sklearn.metrics import precision_recall_curve \
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

In [ ]:
confusion = confusion_matrix(y_test, xgb_preds)
accuracy = accuracy_score(y_test, xgb_preds)
precision = precision_score(y_test, xgb_preds)
recall = recall_score(y_test, xgb_preds)
f1 = f1_score(y_test, xgb_preds)
# ROC-AUC
roc_auc = roc_auc_score(y_test, xgb_pred_probs)
print('오차 행렬')
print(confusion)
# ROc-AUC
print("accuracy : {:.4f}, precision : {:.4f}, recall : {:.4f},\
    F1 : {:.4f}, AUC : {:.4f}".format(accuracy,precision,recall,f1,roc_auc))


### hyperparameter

### General Parameters ( XGBoost 의 어떤 모델을 쓸거야? )
▶ booster [default = 'gbtree'] \
        gbtree : 트리 기반 모델\
        gblinear : 선형 모델\
▶ silent [default = 0]\
        0 : 동작 메세지 프린트 함\
        1 : 동작 메세지 프린트 안함 \
▶ nthread [default = 전체 다 사용]

### Booster Parameters ( 모델의 조건 설정 )
▶ n_estimators [default = 100] : 나무의 개수 (=num_boost_round [default = 10] : 파이썬 래퍼에서 적용)

▶ early_stopping_rounds 
        최대한 몇 개의 트리를 완성해볼 것인지 
        valid loss에 더이상 진전이 없으면 멈춤
        과적합을 방지할 수 있음, n_estimators 가 높을때 주로 사용.

 

▶ learning_rate [default = 0.1] (=eta [default = 0.3] : 파이썬 래퍼에서 적용)
        학습 단계별로 가중치를 얼만큼 사용할지 결정/ 이전의 결과를 얼마나 반영할건지
        낮은 eta -> 낮은 가중치 -> 다음 단계의 결과물 적게 반영 -> 보수적
        일반적으로 0.01 ~ 0.2
        높은 값으로 다른 파라미터 조절하여 결정한 후, 낮춰서 최적의 파라미 결정
        * gradient boost에서는 기울기의 의미, 작으면 꼼꼼히 내려가고 크면 급하게 내려감

▶ min_child_weight [default = 1]
        child 에서 필요한 모든 관측치에 대한 가중치의 최소 합
        이 값보다 샘플 수가 작으면 leaf node가 되는 것
        너무 크면 under-fitting 될 수 있음
        CV로 조절해야함

 

▶ max_depth [default = 6]
        트리의 최대 깊이
        일반적으로 3 ~ 10  
        CV로 조절해야함



▶ gamma [default = 0]
        트리에서 추가적으로 가지를 나눌지를 결정할 최소 손실 감소 값\
        값이 클수록 과적합 감소 효과

 

▶ subsample [default = 1] (=sub_sample : 파이썬 래퍼에서 적용)
        각 트리마다 데이터 샘플링 비율\
        over-fitting 방지
        일반적으로 0.5 ~ 1


▶ colsample_bytree [default = 1]
        각 트리마다 feature 샘플링 비율\
        일반적으로 0.5 ~ 1

▶ reg_lambda [default = 1] (=lambda : 파이썬 래퍼에서 적용)
        L2 regularization(ex. 릿지) 가중치\
        클수록 보수적

▶ reg_alpha [default = 0] (=alpha : 파이썬 래퍼에서 적용)
        L1 regularization(ex. 라쏘) 가중치\
        클수록 보수적
        특성이 매우 많은때 사용해볼만 함

 

▶ scale_pos_weight [default = 1]
        데이터가 불균형할때 사용, 0보다 큰 값\
        보통 값을 음성 데이터 수/ 양성 데이터 수 값으로 함

### Learning Task Parameters ( 모델의 목표 및 계산 방법 설정 )

▶ objective [default = reg:linear] (목적 함수)
- binary:logistic :이진 분류를 위한 로지스틱 회귀, 클래스가 아닌 예측된 확률 반환
- multi:softmax : softmax를 사용한 다중 클래스 분류, 확률이 아닌 예측된 클래스 반환
- multi:softprob : softmax와 같지만 각 클래스에 대한 예상 확률 반환 \
▶ eval_metric [목적 함수에 따라 디폴트 값이 다름(회귀-rmse / 분류-error)]
- rmse : root mean square error
- mae : mean absolute error
- logloss : negative log-likelihood
- error : binary classificaion error rate (임계값 0.5)
- merror : multiclass classification error rate
- mlogloss : multiclass logloss
- auc : area under the curve\
▶ seed [default = 0]
        시드값 고정 (나중에 재현할때 같은 값을 출력하기 위해)

 

## RandomForest Classifier

In [ ]:
model = RandomForestClassifier(n_estimators=50, random_state=0, max_depth = 3)
model.fit(X_train, y_train)

- n_estimators : 모델에서 사용할 트리 갯수(학습시 생성할 트리 갯수)
- criterion : 분할 품질을 측정하는 기능 (default : gini)
- max_depth : 트리의 최대 깊이
- min_samples_split : 내부 노드를 분할하는데 필요한 최소 샘플 수 (default : 2)
- min_samples_leaf : 리프 노드에 있어야 할 최소 샘플 수 (default : 1)
- min_weight_fraction_leaf : min_sample_leaf와 같지만 가중치가 부여된 샘플 수에서의 비율
- max_features : 각 노드에서 분할에 사용할 특징의 최대 수
- max_leaf_nodes : 리프 노드의 최대수
- min_impurity_decrease : 최소 불순도
- min_impurity_split : 나무 성장을 멈추기 위한 임계치
- bootstrap : 부트스트랩(중복허용 샘플링) 사용 여부
- oob_score : 일반화 정확도를 줄이기 위해 밖의 샘플 사용 여부
- n_jobs :적합성과 예측성을 위해 병렬로 실행할 작업 수
- random_state : 난수 seed 설정
- verbose : 실행 과정 출력 여부
- warm_start : 이전 호출의 솔루션을 재사용하여 합계에 더 많은 견적가를 추가
- class_weight : 클래스 가중치

## Lasso Regression

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_set, y_set, test_size=0.2, random_state= 156)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5, random_sate= 156)

In [ ]:
#logit = LogisticRegression(fit_intercept= True, penalty = 'none', solver = 'newton-cg')
lasso = LogisticRegression(fit_intercept= True,  penalty='l1', solver='liblinear', C = 1.0) # LASSO regression

lasso.fit(X_train, y_train)

## Define thresholds with validation set

In [ ]:
result_lasso = pd.DataFrame(lasso.predict_proba(X_val)[:, 1],columns =['pred_prob'])
result_lasso.pred_prob.plot(kind='hist',bins=50, xlim=([0,1]))

In [ ]:
fpr, tpr, thresholds = roc_curve(y_val, result_lasso.pred_prob)
roc_result_lasso = pd.DataFrame([fpr,tpr,thresholds], index=['fpr','tpr','thresholds']).T

In [ ]:
roc_result_lasso['best'] = 0.72*(1-roc_result_lasso['fpr'])+0.5*(roc_result_lasso['tpr']) # (1-fpr) + tpr => 제대로 분류되는 것의 비율을 maximize하는 것. 단 각 항에 가중치 부여.
roc_result_lasso.head()

In [ ]:
optimal_threshold_lasso =roc_result_lasso.thresholds[roc_result_lasso.best.idxmax()]

print("optimal_threshold:",optimal_threshold_lasso)
print("value of purpose function on the optimal_threshold:",roc_result_lasso[roc_result_lasso['thresholds']==optimal_threshold_lasso]['best'])

### Prediction on test set

In [ ]:
result_lasso['test_pred_prob'] = lasso.predict_proba(X_test)[:, 1]

result_lasso.head()

In [ ]:
# create the histogram
plt.hist(result_lasso['test_pred_prob'], bins=10)

# add a vertical line at the cut-off value
plt.axvline(x=optimal_threshold_lasso, color='red', linestyle='--')

# set the x and y axis labels
plt.xlabel('probability')
plt.ylabel('Frequency')

plt.xlim([0, 1])      # X축의 범위: [xmin, xmax]
# show the plot
plt.show()

In [ ]:
result_lasso['lasso_binary'] = np.where(result_lasso.test_pred_prob >= optimal_threshold_lasso, 1, 0)
confusion = confusion_matrix(y_test, result_lasso.lasso_binary)
tp = confusion[1,1] # true positive
tn = confusion[0,0] # true negative
fp = confusion[0,1] # false positive
fn = confusion[1,0] # false negative
sns.heatmap(confusion, annot = True, cmap = 'Blues', fmt='g')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.show()
precision_lasso = tp / (tp + fp) # 정밀도: 모델이 positive라 분류한 것중 실제 값이 positive인 비율
recall_lasso = tp / (tp+ fn) # 재현도: 실제 값이 positive인 것중 모델이 positive라 분류한 비율
f1_score_lasso = tp / (tp + ((fn + fp)/2)) #precision과 recall의 조화평균
accuracy_lasso = (tp + tn) / (tp+fn+fp+tn) # 전체 중 모델이 정확하게 분류한 비율
#risk = (1-tpr)*0.55 + fpr*0.45
print("Precision:", precision_lasso,"\nRecall:\t", recall_lasso,"\nf1_score:", f1_score_lasso,"\nAccuracy:",accuracy_lasso)
#print("Risk:",risk)

In [ ]:
roc_auc_lasso = auc(fpr, tpr)
print('roc_auc:', roc_auc_lasso)

#lpm과 동일한 방법으로 AUROC 그리기

plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area= %0.2f)' % roc_auc_lasso)

##그래프에 직선 추가하기
plt.plot([0,1],[0,1],color='darkorange', linestyle='--')

#x축과 y축 구간 설정하기
plt.xlim([0,1])
plt.ylim([0,1.05])

#축 이름 및 그래프 타이틀, 레전드 추가하기
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic Curve :RandomForest')
plt.legend(loc="lower right")

plt.show()